In [1]:
from benchmark_new import *
%matplotlib inline

In [4]:
helpers = 'pdb'
mcss = '../../../mcss_sizes.pkl'

sp = 'stats21'
xp = 'stats20'

def results(mcss_func, name):
    data = pd.concat(load(version, helpers, mcss) for version in [sp, xp])
    data = add_correct(data, thresh = 2.0)
    data = data[  (data.index.get_level_values('protein') != 'A2AR')
                & (data.index.get_level_values('protein') != 'MAPK14')
                & (data.index.get_level_values('protein') != 'VDR')]
    data.loc[data.index.get_level_values('protein') == 'TRPV1', 'mcss'] = 0
    data = data[mcss_func(data.mcss)]

    ligand = data.xs((helpers, 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                      level=('helpers', 'mode', 'n_ligs', 'features', 'alpha')).filter(regex='rmsd|correct')
    ligand = filter_to_ubiquitous_ligands(ligand)
    target = ligand.groupby(level=list(range(len(ligand.index.levels)-1))).mean().dropna()
    family_target = target.groupby(level=list(range(len(target.index.levels)-1))).mean()
    #family_ligand = ligand.groupby(level=list(range(len(ligand.index.levels)-2))).mean()

    target = pd.merge(target.loc[sp], target.loc[xp], left_index=True, right_index=True)
    family_target = pd.merge(family_target.loc[sp], family_target.loc[xp], left_index=True, right_index=True)

    family_target['SP Top'] = family_target['glide_correct_x']
    family_target['XP Top'] = family_target['glide_correct_y']
    family_target['SP Best'] = family_target['best_correct_x']
    family_target['XP Best'] = family_target['best_correct_y']
    family_target = family_target.filter(regex='P')
    
    family_target['version'] = name
    family_target.reset_index(inplace=True)
    family_target.set_index(['version', 'family'], inplace = True)
    return drug_average(family_target)

pd.concat([results(lambda x: x > -1, 'All'),
           results(lambda x: x >= 0.5, 'Shared scaffold'),
           results(lambda x: x < 0.5, 'No shared scaffold')])

,SP Top,XP Top,SP Best,XP Best
version,,,,
All,0.436061,0.521797,0.801731,0.655895
Shared scaffold,0.472224,0.613316,0.879968,0.802960
No shared scaffold,0.401511,0.421950,0.784473,0.578477
